## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-10-18-53-57 +0000,nypost,Trump reflects on Butler assassination attempt...,https://nypost.com/2025/07/10/us-news/trump-re...
1,2025-07-10-18-44-50 +0000,nypost,Russian stowaway says she snuck onto Paris-bou...,https://nypost.com/2025/07/10/us-news/russian-...
2,2025-07-10-18-41-33 +0000,nyt,US-Brazil Tariffs: What to Know About Trump’s ...,https://www.nytimes.com/2025/07/10/world/ameri...
3,2025-07-10-18-40-27 +0000,nyt,Russia Bombards Ukraine as U.S. Frustration Mo...,https://www.nytimes.com/2025/07/10/world/europ...
4,2025-07-10-18-38-24 +0000,nyt,"Four More Rescued in Red Sea, as Houthis Vow t...",https://www.nytimes.com/2025/07/10/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2302/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,17
32,tariffs,6
129,uk,5
35,russia,5
115,texas,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
32,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,47
0,2025-07-10-18-53-57 +0000,nypost,Trump reflects on Butler assassination attempt...,https://nypost.com/2025/07/10/us-news/trump-re...,40
81,2025-07-10-01-17-00 +0000,wsj,A brewing debate inside the Federal Reserve ov...,https://www.wsj.com/economy/central-banking/wh...,40
56,2025-07-10-12-04-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...,38
18,2025-07-10-17-43-26 +0000,nypost,Secret Service agent from Trump protective det...,https://nypost.com/2025/07/10/us-news/secret-s...,37


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
32,47,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
0,23,2025-07-10-18-53-57 +0000,nypost,Trump reflects on Butler assassination attempt...,https://nypost.com/2025/07/10/us-news/trump-re...
1,23,2025-07-10-18-44-50 +0000,nypost,Russian stowaway says she snuck onto Paris-bou...,https://nypost.com/2025/07/10/us-news/russian-...
17,22,2025-07-10-17-47-40 +0000,nypost,Critics of ex-Houston official who raged at fl...,https://nypost.com/2025/07/10/us-news/ex-houst...
25,19,2025-07-10-16-43-05 +0000,bbc,UK faces rising and unpredictable threat from ...,https://www.bbc.com/news/articles/c98wjzj4jlpo
37,19,2025-07-10-15-01-21 +0000,nypost,Marco Rubio meets Russia’s Sergei Lavrov days ...,https://nypost.com/2025/07/10/us-news/marco-ru...
83,18,2025-07-10-01-07-30 +0000,bbc,Trump threatens Brazil with 50% tariff and dem...,https://www.bbc.com/news/articles/c784ee81y4zo
92,17,2025-07-09-19-53-45 +0000,bbc,Mortgage lending rules change could help first...,https://www.bbc.com/news/articles/cj0mny9pm2do
4,16,2025-07-10-18-38-24 +0000,nyt,"Four More Rescued in Red Sea, as Houthis Vow t...",https://www.nytimes.com/2025/07/10/world/middl...
93,16,2025-07-09-19-48-23 +0000,nyt,Former President Yoon Suk Yeol of South Korea ...,https://www.nytimes.com/2025/07/09/world/asia/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
